In [2]:
import sys
import json
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer
import getpass as gt
import pandas as pd
import numpy as np

from snowflake_conn import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

conn = get_connection()

In [3]:
loc = 'SANDBOX_FFS._tuva_claims.medical_claim'

following clinical logic

In [22]:
# 
sql=f'''
select 
claim_no,
clm_admsn_dt,
clm_thru_dt,
clm_pmt_amt,
nch_bene_dschrg_dt,
clm_drg_cd
from SANDBOX_FFS.LDS.inpatient_base_claim
where clm_admsn_dt <> nch_bene_dschrg_dt
'''
tmp = read_sql(sql)
print(tmp.shape)
tmp.head()

(16785, 6)


,claim_no,clm_admsn_dt,clm_thru_dt,clm_pmt_amt,nch_bene_dschrg_dt,clm_drg_cd
0,-1.000093e+13,20170509,20170510,8545.72,20170510,923
1,-1.000093e+13,20201011,20201021,25223.04,20201021,864
2,-1.000093e+13,20201011,20201021,25223.04,20201021,864
3,-1.000093e+13,20201011,20201021,25223.04,20201021,864
4,-1.000093e+13,20201011,20201021,25223.04,20201021,864


In [25]:
tmp['claim_no'] = tmp['claim_no'].astype(str)
tmp.claim_no.values[:4]

# -10000930038031, -10000930038485

array(['-10000930038031.0', '-10000930038485.0', '-10000930038485.0',
       '-10000930038485.0'], dtype=object)

#### staging
- condition, medical claim, and patient all work
- encounter has no rows because that is created entirely from acute inpatient summary

In [42]:
# 
sql=f'''
select *
from SANDBOX_FFS.core.medical_claim
where claim_id like '-100009300380312020%'
'''
tmp = read_sql(sql)
print(tmp.shape)
tmp 

(1, 39)


,claim_id,claim_line_number,encounter_id,claim_type,patient_id,member_id,claim_start_date,claim_end_date,claim_line_start_date,claim_line_end_date,admission_date,discharge_date,service_category_1,service_category_2,admit_source_code,admit_type_code,discharge_disposition_code,place_of_service_code,bill_type_code,ms_drg_code,apr_drg_code,revenue_center_code,service_unit_quantity,hcpcs_code,hcpcs_modifier_1,hcpcs_modifier_2,hcpcs_modifier_3,hcpcs_modifier_4,hcpcs_modifier_5,rendering_npi,billing_npi,facility_npi,paid_date,paid_amount,allowed_amount,charge_amount,total_cost_amount,data_source,tuva_last_run
0,-10000930038031201760,1,None,institutional,-10000010254653,-10000010254653,None,2017-05-10,2017-05-10,2017-05-10,2017-05-09,2017-05-10,Other,Other,1,3,1,None,111,923,None,0001,0,99024,None,None,None,None,None,9999995399,None,1598783904,None,8545.720000,None,8545.720000,None,medicare_lds,2023-10-06 12:25:34.529269


In [12]:
tmp.groupby('service_category_1')['service_category_2'].value_counts()

service_category_1  service_category_2           
Ancillary           Durable Medical Equipment          1130
Inpatient           Skilled Nursing                    1710
Office Visit        Office Visit                      34339
Other               Other                              2708
Outpatient          Outpatient Hospital or Clinic    133725
                    Urgent Care                        6775
                    Hospice                             943
                    Home Health                         714
                    Emergency Department                 12
Name: count, dtype: int64

In [ ]:
# encounter id

In [ ]:
# int primary dx per claim
sql=f'''
with  __dbt__cte__ed_classification__stg_medical_claim as (

select
    claim_id
    , claim_line_number
    , encounter_id
    , patient_id
    , claim_line_end_date
    , service_category_1
    , service_category_2
    , place_of_service_code
    , revenue_center_code
    , billing_npi
    , facility_npi
    , paid_amount
from SANDBOX_FFS.core.medical_claim
),  __dbt__cte__ed_classification__stg_encounter as (


select
    encounter_id
    , encounter_type
from SANDBOX_FFS.core.encounter
), ed_claims as (
  select
    claim_id
    , sum(med.paid_amount) as claim_paid_amount_sum
  from __dbt__cte__ed_classification__stg_medical_claim med
  left join __dbt__cte__ed_classification__stg_encounter enc
    on med.encounter_id = enc.encounter_id
  where (place_of_service_code = '23' or revenue_center_code in ('0450', '0451', '0452', '0456', '0459', '0981'))
  and (encounter_type <> 'acute inpatient' or encounter_type is null)
  group by claim_id
)

select
   cast(encounter_id as TEXT) as encounter_id
   , cast(claim_id as TEXT) as claim_id
   , cast(patient_id as TEXT) as patient_id
   , cast(normalized_code_type as TEXT) as code_type
   , cast(normalized_code as TEXT) as code
   , cast(condition.normalized_description as TEXT) as description
   , cast(case
            when lower(condition.normalized_description) like '%covid%'
            then condition.normalized_description
            else mapping.ccs_description
           end
          as TEXT) as ccs_description_with_covid
   , recorded_date
   , cast(claim_paid_amount_sum as FLOAT) as claim_paid_amount_sum
from SANDBOX_FFS.ed_classification.ed_classification__stg_condition condition
inner join ed_claims using(claim_id)
left join SANDBOX_FFS.ed_classification._value_set_icd_10_cm_to_ccs mapping 
on condition.normalized_code = mapping.icd_10_cm
where condition_rank = 1
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int merge condition
sql=f'''
ith condition as (
select * from SANDBOX_FFS.ed_classification._int_primary_dx_per_claim
)
, icd9 as (
  select
     icd9 as code
     , edcnnpa
     , edcnpa
     , epct
     , noner
     , injury
     , psych
     , alcohol
     , drug
     , 1 as ed_classification_capture
  from SANDBOX_FFS.ed_classification._value_set_johnston_icd9
)
, icd10 as (
  select
     icd10 as code
     , edcnnpa
     , edcnpa
     , epct
     , noner
     , injury
     , psych
     , alcohol
     , drug
     , 1 as ed_classification_capture
  from SANDBOX_FFS.ed_classification._value_set_johnston_icd10
)

select
   a.*
   , icd10.edcnnpa
   , icd10.edcnpa
   , icd10.epct
   , icd10.noner
   , icd10.injury
   , icd10.psych
   , icd10.alcohol
   , icd10.drug
   , coalesce(icd10.ed_classification_capture, 0) as ed_classification_capture
from condition a
left join icd10
on a.code = icd10.code and a.code_type = 'icd-10-cm'
union all
select
   a.*
   , icd9.edcnnpa
   , icd9.edcnpa
   , icd9.epct
   , icd9.noner
   , icd9.injury
   , icd9.psych
   , icd9.alcohol
   , icd9.drug
   , coalesce(icd9.ed_classification_capture, 0) ed_classification_capture
from condition a
inner join icd9
on a.code = icd9.code and a.code_type = 'icd-9-cm'
'''
tmp = read_sql(sql)
print(tmp.shape)

In [10]:
# int by css
sql=f'''
with summary as (
   select
      classification_order
      , classification_name
      , recorded_date_year
      , ccs_description_with_covid
      , count(distinct(claim_id)) as claim_count
      , sum(claim_paid_amount_sum) as claim_paid_amount_sum
   from SANDBOX_FFS.ed_classification.summary
   group by classification_order, classification_name, recorded_date_year, ccs_description_with_covid
)
, totals_by_classification as (
    select
        classification_name
        , count(distinct(claim_id)) as total_claim_count
        , sum(claim_paid_amount_sum) as total_claim_paid_amount
    from SANDBOX_FFS.ed_classification.summary
    group by
        classification_name
)

select
    summary.*
    ,claim_count/total_claim_count * 100 as percent_claim_count_of_classification
    , claim_paid_amount_sum/total_claim_paid_amount * 100 as percent_claim_paid_amount_sum_of_classification
from summary
inner join totals_by_classification class
    on summary.classification_name = class.classification_name
order by percent_claim_paid_amount_sum_of_classification desc
'''
tmp = read_sql(sql)
print(tmp.shape)

ProgrammingError: 100051 (22012): Division by zero

In [ ]:
# summary
sql=f'''
select
    *
from SANDBOX_FFS.ed_classification._int_condition_with_claim
inner join SANDBOX_FFS.ed_classification._value_set_categories using(classification)
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int by patient race
sql=f'''
with summary as (
  select
     classification_order
     , classification_name
     , recorded_date_year
     , patient_race
     , count(distinct(claim_id)) as claim_count
     , sum(claim_paid_amount_sum) as claim_paid_amount_sum
  from SANDBOX_FFS.ed_classification.summary
  group by classification_order, classification_name, recorded_date_year, patient_race
)
, totals_by_classification as (
    select
        classification_name
        , count(distinct(claim_id)) as total_claim_count
        , sum(claim_paid_amount_sum) as total_claim_paid_amount
    from SANDBOX_FFS.ed_classification.summary
    group by
        classification_name
)

select
    summary.*
    , claim_count/total_claim_count * 100 as percent_claim_count_of_classification
    , claim_paid_amount_sum/total_claim_paid_amount * 100 as percent_claim_paid_amount_sum_of_classification
from summary
inner join totals_by_classification class
    on summary.classification_name = class.classification_name
order by percent_claim_paid_amount_sum_of_classification desc
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int by provider parent organization
sql=f'''
with summary as (
  select
     classification_order
     , classification_name
     , recorded_date_year
     , provider_parent_organization_name_with_provider_name
     , count(distinct(claim_id)) as claim_count
     , sum(claim_paid_amount_sum) as claim_paid_amount_sum

  from SANDBOX_FFS.ed_classification.summary
  group by classification_order, classification_name, recorded_date_year, provider_parent_organization_name_with_provider_name
)
, totals_by_classification as (
    select
        classification_name
        , count(distinct(claim_id)) as total_claim_count
        , sum(claim_paid_amount_sum) as total_claim_paid_amount
    from SANDBOX_FFS.ed_classification.summary
    group by
        classification_name
)
select
    summary.*
    , claim_count/total_claim_count * 100 as percent_claim_count_of_classification
    , claim_paid_amount_sum/total_claim_paid_amount * 100 as percent_claim_paid_amount_sum_of_classification
from summary
inner join totals_by_classification class
    on summary.classification_name = class.classification_name
order by percent_claim_paid_amount_sum_of_classification desc
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int by provider practice state
sql=f'''
with summary as (
  select
     classification_order
     , classification_name
     , recorded_date_year
     , provider_practice_state
     , count(distinct(claim_id)) as claim_count
     , sum(claim_paid_amount_sum) as claim_paid_amount_sum

  from SANDBOX_FFS.ed_classification.summary
  group by classification_order, classification_name, recorded_date_year, provider_practice_state
)
, totals_by_classification as (
    select
        classification_name
        , count(distinct(claim_id)) as total_claim_count
        , sum(claim_paid_amount_sum) as total_claim_paid_amount
    from SANDBOX_FFS.ed_classification.summary
    group by
        classification_name
)
select
    summary.*
    , claim_count/total_claim_count * 100 as percent_claim_count_of_classification
    , claim_paid_amount_sum/total_claim_paid_amount * 100 as percent_claim_paid_amount_sum_of_classification
from summary
inner join totals_by_classification class
    on summary.classification_name = class.classification_name
order by percent_claim_paid_amount_sum_of_classification desc
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int by year month
sql=f'''
with summary as (
   select
      classification_order
      , classification_name
      , recorded_date_year
      , recorded_date_year_month
      , count(distinct(claim_id)) as claim_count
      , sum(claim_paid_amount_sum) as claim_paid_amount_sum
   from SANDBOX_FFS.ed_classification.summary
   group by classification_order, classification_name, recorded_date_year, recorded_date_year_month
)
, totals_by_classification as (
    select
        classification_name
        , count(distinct(claim_id)) as total_claim_count
        , sum(claim_paid_amount_sum) as total_claim_paid_amount
    from SANDBOX_FFS.ed_classification.summary
    group by
        classification_name
)
select
    summary.*
    , claim_count/total_claim_count * 100 as percent_claim_count_of_classification
    , claim_paid_amount_sum/total_claim_paid_amount * 100 as percent_claim_paid_amount_sum_of_classification
from summary
inner join totals_by_classification class
    on summary.classification_name = class.classification_name
order by classification_order, recorded_date_year_month desc
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int by year
sql=f'''
select
   classification_order
   , classification_name
   , recorded_date_year
   , count(distinct(claim_id)) as claim_count
   , sum(claim_paid_amount_sum) as claim_paid_amount_sum
from SANDBOX_FFS.ed_classification.summary
group by classification_order, classification_name, recorded_date_year
order by recorded_date_year, classification_order
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int condition with claim
sql=f'''
/*
Denormalized view of each condition row with additional provider and patient level
information merged on based on the header level detail on the claim
*/



with  __dbt__cte__ed_classification__stg_medical_claim as (


select
    claim_id
    , claim_line_number
    , encounter_id
    , patient_id
    , claim_line_end_date
    , service_category_1
    , service_category_2
    , place_of_service_code
    , revenue_center_code
    , billing_npi
    , facility_npi
    , paid_amount
from SANDBOX_FFS.core.medical_claim
),  __dbt__cte__ed_classification__stg_patient as (


select
    patient_id
    , sex
    , birth_date
    , race
    , state
from SANDBOX_FFS.core.patient
), deduped_claims as (
   select distinct
      claim_id
      , claim_line_end_date
      , service_category_1
      , service_category_2
      , facility_npi
      , billing_npi
      , patient_id
   from __dbt__cte__ed_classification__stg_medical_claim mc
   where mc.claim_line_number = 1
)
, deduped_providers as (
   select distinct
      npi
      , entity_type_code
      , entity_type_description
      , primary_taxonomy_code
      , primary_specialty_description
      , provider_first_name
      , provider_last_name
      , provider_organization_name
      , parent_organization_name
      , practice_city
      , practice_state
      , practice_zip_code
   from SANDBOX_FFS.terminology.provider
)

select
  c.claim_id
  , c.patient_id
  , c.recorded_date
  , c.code
  , c.description
  , c.ccs_description_with_covid
  , c.recorded_date_year
  , c.recorded_date_year_month
  , c.claim_paid_amount_sum
  , c.classification
  -- claim level additions
  , mc.service_category_1
  , mc.service_category_2
  -- provider level additions
  , coalesce(fp.parent_organization_name
    , bp.parent_organization_name
    , fp.provider_first_name||' '||fp.provider_last_name
    , bp.provider_first_name||' '||bp.provider_last_name
    , fp.provider_organization_name
    , bp.provider_organization_name) as provider_parent_organization_name_with_provider_name
  , coalesce(fp.provider_first_name, bp.provider_first_name) as provider_first_name
  , coalesce(fp.provider_last_name, bp.provider_last_name) as provider_last_name
  , coalesce(fp.provider_organization_name, bp.provider_organization_name) as provider_organization_name
  , coalesce(fp.parent_organization_name, bp.parent_organization_name) as provider_parent_organization_name
  , coalesce(fp.practice_state, bp.practice_state) as provider_practice_state
  , coalesce(fp.practice_zip_code, bp.practice_zip_code) as provider_practice_zip_code
  -- patient level additions
  , p.sex as patient_gender
  , p.birth_date as patient_birth_date
  , floor(datediff(
        hour,
        birth_date,
        current_date
        ) / 8766.0) as patient_age
  , p.race as patient_race
  , p.state as patient_state

from SANDBOX_FFS.ed_classification._int_condition_with_class c
inner join deduped_claims mc
      on c.claim_id = mc.claim_id
      and c.recorded_date = mc.claim_line_end_date
left join deduped_providers fp on mc.facility_npi = fp.npi
left join deduped_providers bp on mc.billing_npi = bp.npi
left join __dbt__cte__ed_classification__stg_patient p on mc.patient_id = p.patient_id
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int condition with class
sql=f'''
select
   a.encounter_id
   , a.claim_id
   , a.patient_id
   , a.code_type
   , a.code
   , a.description
   , a.ccs_description_with_covid
   , a.recorded_date
   , cast(date_part('year', recorded_date) as TEXT) as recorded_date_year
   , cast(date_part('year', recorded_date) as TEXT)
     || lpad(cast(date_part('month', recorded_date) as TEXT), 2, '0')
     as recorded_date_year_month
   , a.claim_paid_amount_sum
   , case greatest(edcnnpa, edcnpa, epct, noner, injury, psych, alcohol, drug)
          when edcnnpa then 'edcnnpa'
          when edcnpa then 'edcnpa'
          when epct then 'epct'
          when noner then 'noner'
          when injury then 'injury'
          when psych then 'psych'
          when alcohol then 'alcohol'
          when drug then 'drug'
          else 'unclassified'
   end as classification
from SANDBOX_FFS.ed_classification._int_merge_condition a
where ed_classification_capture = 1
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# int patient list
sql=f'''
with __dbt__cte__ed_classification__stg_patient as (


select
    patient_id
    , sex
    , birth_date
    , race
    , state
from SANDBOX_FFS.core.patient
) select
   p.*
   , c.claim_id
   , c.recorded_date
   , c.recorded_date_year
   , c.recorded_date_year_month
   , c.code_type
   , c.code
   , c.description
   , c.ccs_description_with_covid
   , ec.classification_order
   , c.classification
   , ec.classification_name
from SANDBOX_FFS.ed_classification._int_condition_with_class c
inner join __dbt__cte__ed_classification__stg_patient p using(patient_id)
inner join SANDBOX_FFS.ed_classification._value_set_categories ec using(classification)
'''
tmp = read_sql(sql)
print(tmp.shape)

In [ ]:
# 
sql=f'''

'''
tmp = read_sql(sql)
print(tmp.shape)